En este apartado, llevaremos a cabo transformaciones a los datasets limpios:

- australian_user_reviews_cleaned.csv
- australian_users_items_cleaned.csv
- output_steam_games_cleaned.csv

El objetivo es tener archivos con la información necesaria para crear las funciones que va a consumir la API de acuerdo a los requerimientos del proyecto.

### 1. Importación de las bibliotecas necesarias

In [2]:
import pandas as pd
import numpy as np

### 2. Carga de Datasets

In [2]:
df_UserReviews = pd.read_csv("../Archivos_Limpios/australian_user_reviews_cleaned.csv")
df_UsersItems = pd.read_csv("../Archivos_Limpios/australian_users_items_cleaned.csv")
df_steam_games = pd.read_csv("../Archivos_Limpios/output_steam_games_cleaned.csv")

### 3. Transformaciones

#### - PlayTimeGenre

In [3]:
# Sumamos el número de horas por item_id en el DataFrame df_UsersItems
df_UsersItemsGroup = df_UsersItems.groupby("item_id")["hours_game"].sum().reset_index()

# Realizamos un merge entre los DataFrames: df_steam_games y df_UsersItemsGroup
play_time_genre = pd.merge(df_steam_games[["item_id", "genres", "year"]], df_UsersItemsGroup[["item_id","hours_game"]], on="item_id")

#Agrupamos por genero y año
play_time_genre = play_time_genre.groupby(["year", "genres"])["hours_game"].sum().reset_index()

# Encontramos el género con más horas jugadas por año 
play_time_genre = play_time_genre.loc[play_time_genre.groupby(["year", "genres"])["hours_game"].idxmax()]

# Ordenamos el DataFrame resultante
play_time_genre = play_time_genre.sort_values(by=["genres", "hours_game", "year"], ascending=[True, False, True]).reset_index(drop=True)

In [4]:
# Guardamos el resultado en un archivo CSV
play_time_genre.to_csv("PlayTimeGenre.csv", index=False)

#### - UserForGenre

In [5]:
# Creamos un DataFrame a partir de df_steam_games, eliminando algunas columnas 
df_game = df_steam_games.drop(["name", "developer", "price"],axis=1)

# Realizamos un merge entre los DataFrames: df_UsersItems y df_game
merged_data=pd.merge(df_UsersItems,df_game, left_on="item_id", right_on="item_id", how='right')

# Eliminamos la columna "item_id"
merged_data.drop(["item_id"], axis=1, inplace=True)
merged_data.dropna(inplace=True)

# Tranformamos el tipo de dato de las columnas "hours_game" y "year"
merged_data["hours_game"] = merged_data["hours_game"].astype(int)
merged_data["year"] = merged_data["year"].astype(int)

# Encontramos el jugador con más horas jugadas por género y año
user_for_genre = merged_data.loc[merged_data.groupby(["genres", "year"])["hours_game"].idxmax()]

# Filtramos las columnas necesarias
user_for_genre = user_for_genre[["genres", "year", "user_id", "hours_game"]]

# Agrupamos para obtener las horas jugadas por año para el usuario seleccionado
user_for_genre = user_for_genre.groupby(["genres", "year", "user_id"])["hours_game"].sum().reset_index()

In [6]:
# Guardamos el resultado en un archivo CSV
user_for_genre.to_csv("UserForGenre.csv", index=False)

#### - UsersRecommend

In [3]:
# Seleccionamos las columnas que necesitaremos del DataFrame df_UserReviews
columns_UserReviews = ["item_id", "recommend", "sentiment_analysis", "year"]

# Seleccionamos las columnas que necesitaremos del DataFrame df_steam_games
columns_SteamGames = ["item_id", "name"]

# Creamos dos nuevos DataFrames a partir de esaas selecciones de columnas
df_UserReviews_selected = df_UserReviews[columns_UserReviews]

df_SteamGames_selected = df_steam_games[columns_SteamGames]

# Realizamos un merge entre los DataFrames: df_UserReviews_selected y df_SteamGames_selected
df_merged = pd.merge(df_UserReviews_selected, df_SteamGames_selected, on="item_id")

# Filtramos los registros que cumplan algunas condiciones
result = df_merged.loc[(df_merged["recommend"] == True) & (df_merged["sentiment_analysis"].isin([1, 2])), ["year", "name"]]

# Agrupamos por las columnas "year" y "name", contando las ocurrencias
users_recommend = result.groupby(["year", "name"]).size().reset_index(name="count")

# Ordenamos por las columnas "year" y "count" en orden descendente
users_recommend = users_recommend.sort_values(by=["year", "count"], ascending=[False, False])

# Obteneemos el top 3 por año
users_recommend = users_recommend.groupby("year").head(3)

In [4]:
# Guardamos el resultado en un archivo CSV
users_recommend.to_csv("UsersRecommend.csv", index=False)

#### - UsersWorstDeveloper

In [3]:
# Realizamos un merge entre los DataFrames: df_UserReviews y df_steam_games
df_merged = pd.merge(df_UserReviews, df_steam_games[["item_id", "developer"]], on="item_id")

# Filtramos los registros que cumplan algunas condiciones
result = df_merged.loc[(df_merged["recommend"] == False) & (df_merged["sentiment_analysis"] == 0), ["year", "developer"]]

# Agrupamos por las columnas "year" y "developer", contando las ocurrencias
users_worst_developer = result.groupby(["year", "developer"]).size().reset_index(name="count")

# Ordenamos por las columnas "year" y "count" en orden descendente
users_worst_developer = users_worst_developer.sort_values(by=["year", "count"], ascending=[False, False])

# Obtenemos el top 3 por año
users_worst_developer = users_worst_developer.groupby("year").head(3)  

In [5]:
# Guardamos el resultado en un archivo CSV
users_worst_developer.to_csv("UsersWorstDeveloper.csv", index=False)

#### - sentiment_analysis

In [3]:
# Realizamos un merge entre los DataFrames: df_UserReviews y df_steam_games
df_merged = pd.merge(df_UserReviews[["item_id", "sentiment_analysis"]], df_steam_games[["item_id", "developer"]], on="item_id")

# Agrupamos por las columnas "year" y "developer", contando las ocurrencias
sentiment_analysis = df_merged.groupby(["developer", "sentiment_analysis"]).size().reset_index(name="count")

# Pivoteamos el DataFrame para obtener sentiment_analysis como columnas según su valor
sentiment_analysis = sentiment_analysis.pivot(index="developer", columns="sentiment_analysis", values="count").reset_index()

# Rellenamos los NaN con 0
sentiment_analysis = sentiment_analysis.fillna(0)

# Renombramos las columnas
sentiment_analysis.columns = ["developer", "Negative", "Neutral", "Positive"]

# Convertimos las columnas "Negative", "Neutral" y "Positive" a tipo int
sentiment_analysis[["Negative", "Neutral", "Positive"]] = sentiment_analysis[["Negative", "Neutral", "Positive"]].astype(int)

In [4]:
# Guardamos el resultado en un archivo CSV
sentiment_analysis.to_csv("SentimentAnalysis.csv", index=False)